## This notebook shows how to use Orchestrator APIs for user experiments

In [ ]:
import os
from fabric_cm.credmgr.credmgr_proxy import CredmgrProxy
from fabric_cf.orchestrator.orchestrator_proxy import OrchestratorProxy
import json

In [ ]:
credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
credmgr_proxy = CredmgrProxy(credmgr_host=credmgr_host)
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
orchestrator_proxy = OrchestratorProxy(orchestrator_host=orchestrator_host)

### Fabric Tokens
Fabric has 2 kinds of tokens:<br>
- Identity : required for Control/Measurement Framework APIs. Identity Token is valid upto an hour.
- Refresh : required to generate new Identity Tokens valid. Refresh Token is valid for 24 hours.

Fabric Identity token is required for Control/Measurement Framework APIs.<br>

When user logins to Jupyterhub after authenticating against CILogon, OIDC refresh token is derived. <br>
This token is available as the environment variable `CILOGON_REFRESH_TOKEN`.<br>

On the first login, we use `CILOGON_REFRESH_TOKEN` to generate new Fabric Identity Token and Fabric Refresh Token.<br>
For any subsequent use, we use Fabric Refresh Token. On every refresh, Fabric Refresh Token is changed and updated.<br>

NOTE: These steps are required for any experiments on Fabric Testbed.<br>

In [ ]:
not_found=False
fabric_refresh_token=None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
print("Fabric Refresh Token {}".format(fabric_refresh_token))
print("CILOGON_REFRESH_TOKEN environment variable: {}".format(os.environ['CILOGON_REFRESH_TOKEN']))

### Get new Fabric Identity Token and update Fabric Refresh Token

Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters in the refresh call below.

In [ ]:
try:
    refresh_res = credmgr_proxy.refresh(project_name='all', scope='all', refresh_token=fabric_refresh_token)
    print("New Tokens received: {}".format(json.dumps(refresh_res)))
    fabric_id_token=refresh_res['id_token']
    fabric_refresh_token=refresh_res['refresh_token']
    print()
    print("New Refresh Token: {}".format(fabric_refresh_token))
    print()
    print("Stored new Refresh Token")
    %store fabric_refresh_token
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

### Orchestrator API example to query for available resources

In [ ]:
status, advertised_topology = orchestrator_proxy.resources(token=fabric_id_token)

print(f"Status: {status}")
print(f"Toplogy: {advertised_topology}")

In [ ]:
advertised_topology.draw()

## Create Slice

In [ ]:
import fim.user as fu
# Create topology
t = fu.ExperimentTopology()

# Add node
n1 = t.add_node(name='n1', site='RENC')

# Set capacities
cap = fu.Capacities()
cap.set_fields(core=4, ram=64, disk=500)

# Set Properties
n1.set_properties(capacities=cap, image_type='qcow2', image_ref='default_ubuntu_20')

# Add PCI devices
n1.add_component(ctype=fu.ComponentType.SmartNIC, model='ConnectX-5', name='nic1')

# Add node
n2 = t.add_node(name='n2', site='RENC')

# Set properties
n2.set_properties(capacities=cap, image_type='qcow2', image_ref='default_ubuntu_20')

# Add PCI devices
n2.add_component(ctype=fu.ComponentType.SmartNIC, model='ConnectX-6', name='nic2')




# # Add node
# n3 = t.add_node(name='n3', site='LBNL')

# # Set properties
# n3.set_properties(capacities=cap, image_type='qcow2', image_ref='default_centos_8')

# # Add PCI devices
# n3.add_component(ctype=fu.ComponentType.GPU, model='Tesla T4', name='nic3')

# Generate Slice Graph
slice_graph = t.serialize()

ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read()
    ssh_key=ssh_key.strip()

# Request slice from Orchestrator
status, reservations = orchestrator_proxy.create(token=fabric_id_token, slice_name='JupyterSlice3', slice_graph=slice_graph, ssh_key=ssh_key)

print("Response Status {}".format(status))
print("Reservations created {}".format(reservations))

In [ ]:
# Set the Slice ID from output of the above command
slice_id=reservations[0].slice_id

## Query Slices

In [ ]:
status, slices = orchestrator_proxy.slices(token=fabric_id_token)

print("Response Status {}".format(status))
print("Slices {}".format(slices))

## Query Slice

In [ ]:
status, slice_obj = orchestrator_proxy.get_slice(token=fabric_id_token, slice_id=slice_id)

print("Response Status {}".format(status))
print("Response received {}".format(slice_obj))

In [ ]:
slice_obj.draw()

## Query Slivers

In [ ]:
status, reservations = orchestrator_proxy.slivers(token=fabric_id_token, slice_id=slice_id)

print("Response Status {}".format(status))
print("Reservations {}".format(reservations))

In [ ]:
# Set the Sliver ID from output of the above command by capturing reservation_id
sliver_id=reservations[0].reservation_id

## Query Sliver

In [ ]:
status, reservation = orchestrator_proxy.slivers(token=fabric_id_token, slice_id=slice_id, sliver_id=sliver_id)

print("Response Status {}".format(status))
print("Reservation {}".format(reservation))

## Slice Status

In [ ]:
status, slice_status = orchestrator_proxy.slice_status(token=fabric_id_token, slice_id=slice_id)

print("Response Status {}".format(status))
print("Slice Status {}".format(slice_status))

## Sliver Status

In [ ]:
status, reservation_status = orchestrator_proxy.sliver_status(token=fabric_id_token, slice_id=slice_id, sliver_id=sliver_id)

print("Response Status {}".format(status))
print("Reservation Status {}".format(reservation_status))

## Work on the servers start from here

We are going to use paramiko to control our servers through ssh.

In [ ]:
import paramiko

We have two servers with two IPs. Let's print them.

In [ ]:
reservations[0].management_ip

In [ ]:
reservations[1].management_ip

Below, we connect to our first server.

In [ ]:
key = paramiko.RSAKey.from_private_key_file("/home/fabric/.ssh/id_rsa")
client1 = paramiko.SSHClient()
client1.load_system_host_keys()
client1.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())

client1.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client1.connect(reservations[0].management_ip,username='ubuntu',pkey = key)

client1

Connection should be established. Let's issue some test commands:

In [ ]:
stdin, stdout, stderr = client1.exec_command('lsb_release -a')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client1.exec_command('ip addr')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client1.exec_command('curl https://ipinfo.io/ip')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Now we connect to our second server.

In [ ]:
key = paramiko.RSAKey.from_private_key_file("/home/fabric/.ssh/id_rsa")
client2 = paramiko.SSHClient()
client2.load_system_host_keys()
client2.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())

client2.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client2.connect(reservations[1].management_ip,username='ubuntu',pkey = key)

client2

Again, some test commands.

In [ ]:
stdin, stdout, stderr = client2.exec_command('curl https://ipinfo.io/ip')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client1.exec_command('curl https://ipinfo.io/ip')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

### We have client1 connected to the machine at 152.54.15.44, and client2 connected to the other machine at 152.54.15.51.

In [ ]:
client1ip = reservations[0].management_ip
client1ip

In [ ]:
client2ip = reservations[1].management_ip
client2ip

### Now, let's do a quick ping experiment. Let's let each of the servers ping the other server.

In [ ]:
stdin, stdout, stderr = client2.exec_command('ping -c 10 ' + client1ip)
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client1.exec_command('ping -c 10 ' + client2ip)
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

### Ping experiment complete. Both nodes can see each other.

### Now, let's start a Kubernetes cluster on those two nodes.

We follow the instructions that we have here: https://github.com/apache/openwhisk-deploy-kube/blob/master/docs/k8s-diy-ubuntu.md

Let's let the node connected to client 1 be master. We now simply follow the instructions one by one.

In [ ]:
stdin, stdout, stderr = client1.exec_command('sudo apt update')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client1.exec_command('sudo apt-get install -y docker.io')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client1.exec_command('sudo apt-get update && sudo apt-get install -y apt-transport-https curl')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client1.exec_command('curl -s https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
# shell = client1.invoke_shell()
# shell.send('cat <<EOF | sudo tee /etc/apt/sources.list.d/kubernetes.list\n')
# shell.send('deb https://apt.kubernetes.io/ kubernetes-xenial main\n')
# shell.send('EOF\n')


stdin, stdout, stderr = client1.exec_command('cat <<EOF | sudo tee /etc/apt/sources.list.d/kubernetes.list\ndeb https://apt.kubernetes.io/ kubernetes-xenial main\nEOF\n')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client1.exec_command('cat /etc/apt/sources.list.d/kubernetes.list')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client1.exec_command('sudo apt-get update')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client1.exec_command('sudo apt-get install -y kubelet kubeadm kubectl')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client1.exec_command('sudo apt-mark hold kubelet kubeadm kubectl')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client1.exec_command('sudo swapoff -a')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

The setup part is complete. Now, we issue the kubeadm init command. We do it like this:

In [ ]:
stdin, stdout, stderr = client1.exec_command('sudo kubeadm init --pod-network-cidr=192.168.0.0/16 --apiserver-advertise-address=0.0.0.0')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

As stated, the initialization was successful. Now we need to save the join command somewhere, because we will need to use it later in the client.

In [ ]:
stdin, stdout, stderr = client1.exec_command('mkdir -p $HOME/.kube')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client1.exec_command('sudo cp -i /etc/kubernetes/admin.conf $HOME/.kube/config')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client1.exec_command('sudo chown $(id -u):$(id -g) $HOME/.kube/config')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client1.exec_command('kubectl apply -f https://docs.projectcalico.org/v3.9/manifests/calico.yaml')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client1.exec_command('kubectl get nodes')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

We need to wait for just a few more seconds for the STATUS to change to Ready.

In [ ]:
stdin, stdout, stderr = client1.exec_command('kubectl get nodes')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

It's now ready.

### We are now done with the server. Let's do the client. Remember, we are following the instructions in the link that we have previously mentioned.

In [ ]:
stdin, stdout, stderr = client2.exec_command('sudo apt-get update')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client2.exec_command('sudo apt-get install -y docker.io')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client2.exec_command('sudo apt-get update && sudo apt-get install -y apt-transport-https curl')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client2.exec_command('curl -s https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client2.exec_command('cat <<EOF | sudo tee /etc/apt/sources.list.d/kubernetes.list\ndeb https://apt.kubernetes.io/ kubernetes-xenial main\nEOF\n')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client2.exec_command('cat /etc/apt/sources.list.d/kubernetes.list')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client2.exec_command('sudo apt-get update')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client2.exec_command('sudo apt-get install -y kubelet kubeadm kubectl')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client2.exec_command('sudo apt-mark hold kubelet kubeadm kubectl')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client2.exec_command('sudo swapoff -a')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

### The setup part is complete. Now we need to do the join command. We do it like this:

As a note. When we do "kubeadm init" in the server, it prints the join command for us. What we do is that we copy it and paste it in the client. And don't forget to add sudo.

In [ ]:
stdin, stdout, stderr = client2.exec_command('sudo kubeadm join 10.20.4.228:6443 --token 8rtm7z.e26gfxwe669sgngk --discovery-token-ca-cert-hash sha256:19dc666a1c8fb4bce98dbb2019cdaa6e6b5390209e011db124d82cd84e9499b8')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Let's check back at the server.

In [ ]:
stdin, stdout, stderr = client1.exec_command('kubectl get nodes')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

### We can see that the worker node is connected. Task complete.

### The next task is to deploy a hello world application.

First, we pull a hello world image and create a "deployment".

In [ ]:
stdin, stdout, stderr = client1.exec_command('kubectl create deployment kubernetes-bootcamp --image=gcr.io/google-samples/kubernetes-bootcamp:v1')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Now let's do some status commands. They just show us some information.

In [ ]:
stdin, stdout, stderr = client1.exec_command('kubectl get pods --all-namespaces')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

In [ ]:
stdin, stdout, stderr = client1.exec_command('kubectl describe pod kubernetes-bootcamp-57978f5f5d-4lpvq')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

The next thing we need to do is to create what is called a "service". We are going to use it to expose the deployment to the outside, through a port, which is 8080. Like this:

_Note that the service itself will still need to be exposed. There's another "expose" step that we need to make._

In [ ]:
stdin, stdout, stderr = client1.exec_command('kubectl expose deployment/kubernetes-bootcamp --type="ClusterIP" --port 8080')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Let's check that the service was created.

In [ ]:
stdin, stdout, stderr = client1.exec_command('kubectl get service kubernetes-bootcamp')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Finally, we need to run a port forwarding command in order to, as we said, expose the service to the outside.

In [ ]:
stdin, stdout, stderr = client1.exec_command('kubectl port-forward --address 0.0.0.0 service/kubernetes-bootcamp 8080:8080 > /dev/null 2>&1 &')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Now our application should finally be visible. Let's test the deployment locally, on the master machine itself.

In [ ]:
stdin, stdout, stderr = client1.exec_command('curl localhost:8080')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

It does work. Now let's test it remotely. From our notebook.

In [ ]:
import requests

x = requests.get('http://152.54.15.36:8080') #, verify=False)
print(x.status_code)
print(x.content)

We have our result. Experiment complete.

## Delete Slice

You can now delete the slice. The code below should do it.

In [ ]:
# status, result = orchestrator_proxy.delete(token=fabric_id_token, slice_id=slice_id)

# print("Response Status {}".format(status))
# print("Response received {}".format(result))